In [ ]:
# Assignment 2 - 5)  Naive base with Binomial feature

import urllib2
import random
import math
import numpy as np
from math import log
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix

def nCr_prob(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)
    
def separateDataByClass(datas, target):
    
    class1 = []
    class2 = []

    for i in range(target.shape[0]):
        if target[i] == 0.0:
            class1.append(np.array(datas[i]))
        else:
            class2.append(np.array(datas[i]))

    return np.array(class1),np.array(class2)

def membership_function(test_set, no_of_words, alpha, priorVal):
    
    gX = []
    for j in range(test_set.shape[0]):  
        value_1 = sum([(math.log(nCr_prob(int(no_of_words[j]),int(test_set[j,i])))) for i in range(test_set.shape[1])])
        value_2 = sum([(test_set[j,i] * math.log(alpha[i])) for i in  range(test_set.shape[1])])
        value_3 = sum([((no_of_words[j] - test_set[j,i]) * math.log(1-alpha[i])) for i in range(test_set.shape[1])])

        total = value_1 + value_2 + value_3 + math.log(priorVal)
        gX.append(total)

    return gX

def discriminant_function(g1X, g2X):
    
    predicted = []
    diff = [g2X[i] - g1X[i] for i in range(len(g1X))]
    
    for idx in diff:
        if idx > 0:
            predicted.append(1.0)
        else:
            predicted.append(0.0)
    
    return predicted

def confusion_matrixx(y_test_data, predicted_y, total_class):

    true_positive=0
    true_negative=0
    false_positive=0
    false_negative = 0
    
    conf_matrix = np.zeros((len(total_class),len(total_class)))
    
    for i in range(0, len(y_test_data)):
        if(y_test_data[i]==total_class[0] and (predicted_y[i]==total_class[0])):
            true_positive = true_positive + 1
        elif(y_test_data[i]==total_class[1] and (predicted_y[i]==total_class[1])):
            true_negative = true_negative + 1    
        elif(y_test_data[i]==total_class[0] and (predicted_y[i]==total_class[1])):
            false_positive  = false_positive + 1
        elif(y_test_data[i]==total_class[1] and (predicted_y[i]==total_class[0])):
            false_negative = false_negative + 1 
            
    conf_matrix[0,0]= true_positive
    conf_matrix[0,1]= false_positive
    conf_matrix[1,0]= false_negative
    conf_matrix[1,1]= true_negative
    
    return conf_matrix

def findOtherParameters(confusion_mat):
    
    list_diagonal = np.zeros(confusion_mat.shape[0])
    list_row_sum = np.zeros(confusion_mat.shape[0])
    list_column_sum=np.zeros(confusion_mat.shape[1])
    
    precision_value = []
    recall_value = []
    f_measure_value = []
    
    total = np.sum(confusion_mat)
    confuse_diagonal = 0

    for i in range(confusion_mat.shape[0]):
        for j in range(confusion_mat.shape[1]):
            list_row_sum[i] += confusion_mat[i][j]
            list_column_sum[i] += confusion_mat[j][i]
            if(i==j):
                list_diagonal[i] = confusion_mat[i][j]
                confuse_diagonal +=  confusion_mat[i][j]
    print "Accuracy", float(confuse_diagonal)/total
    accuracy = float(confuse_diagonal)/total
    
    for index in range(len(list_row_sum)):
        if list_row_sum[index]==0:
            precision_value.append(0.0)
        else:
            precision_value.append((float)(list_diagonal[index]) / list_row_sum[index])
            
        if list_column_sum[index]==0:
            recall_value.append(0)
        else:
            recall_value.append((float)(list_diagonal[index]) / list_column_sum[index])
        
        if precision_value[index]==0 or recall_value[index]==0:
            f_measure_value.append(0)
        else:
            f_measure_value.append((float) (2 * precision_value[index] * recall_value[index]) / (precision_value[index] + recall_value[index]))
        
    return accuracy, precision_value, recall_value, f_measure_value

def main():
    
    temp = 0
    final_precision=0
    final_recall = 0
    final_fmeasure = 0
    final_accuracy = 0
    
    data = urllib2.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data")
    dataset = []
    target = []
    
    files = np.loadtxt(data, dtype=float, delimiter=',')
    dataset = np.array(files[:,:54])
    dataset = map(lambda x: x * 100, dataset)
    dataset = np.array(dataset)
    
    print len(dataset[0])
    target = np.array(files[:,-1], dtype='float')

#    print len(dataset[0])
    print "\nK-Fold started "        
    kf = KFold(len(files), n_folds=10, shuffle=False)
    print "No of folds = ",len(kf)

    for train_index, test_index in kf:
        
        trainingData, trainingTarget = dataset[train_index], target[train_index]
        testinggData, testingTarget = dataset[test_index], target[test_index]

        # seperate data by class
        class_1, class_2 = separateDataByClass(trainingData, trainingTarget)

        c1_length = len(class_1)
        c2_length = len(class_2)

        # find prior array
        prior_array = [(c1_length*1.0/(c1_length+c2_length)), (c2_length*1.0/(c1_length+c2_length))]
        print prior_array

        # find alpha and membership function for class_1
        no_of_words1 = [sum(dataset[i]) for i in range(class_1.shape[0])]
        alpha_1  = (list((np.sum(class_1[:,i])/sum(no_of_words1)) for i in range(class_1.shape[1])))
        g1X = membership_function(testinggData, no_of_words1, alpha_1, prior_array[0])
        
        # find alpha and membership function for class_2
        no_of_words2 = [sum(dataset[i]) for i in range(class_2.shape[0])]
        alpha_2  = (list((np.sum(class_2[:,i])/sum(no_of_words2)) for i in range(class_2.shape[1])))
        g2X = membership_function(testinggData, no_of_words2, alpha_2, prior_array[1])

        # find discriminanet function
        disc_function = discriminant_function(g1X, g2X)

        # convert data array to list
        print type(testingTarget)
        Y_Test_Data = testingTarget.tolist()
        
        #find confusion matrix
        no_of_class = [0, 1]
        #confusion_mat = confusion_matrix(Y_Test_Data, disc_function)
        confusion_mat = confusion_matrixx(Y_Test_Data,disc_function,no_of_class)
        print confusion_mat
        # find precision, recall and f_measure
        accuracy, precision_val, recall_val, f_measure_val = findOtherParameters(confusion_mat)
     
        print "Precision = ", precision_val
        print "Recall = ", recall_val
        print "F-Measure", f_measure_val
        print "\n Fold Completed \n"
        
        if accuracy>temp:
            final_accuracy = accuracy
            final_precision = precision_val
            final_recall = recall_val
            final_fmeasure = f_measure_val
            temp = accuracy

        print "\n K-fold completed \n"
    
        break
        
    # print errors
    print "Final Output : "
    print "Accuracy = ", final_accuracy
    print "Precision = ", final_precision
    print "Recall = ", final_recall
    print "F-Measure", final_fmeasure
    print "\n Fold Completed \n"
        
main()